# Applying Indicators to News Events
Takes the output from `iq_feed_cleaning.ipynb` and for each row, adds indicators. \
This is preferred over calculating the indicators for each timestamp/date, as it saves 
a lot of computations. \
It does require us to do a look-up, but so does the alternative.

In [ ]:
# Sort by stocks
# Sort by time
# Add stock specific indicator
# Add global indicator (S&P500)